In [66]:
import requests
from bs4 import BeautifulSoup
from IPython.display import display , Markdown
from sklearn.feature_extraction.text import TfidfVectorizer
import re

r.text is the content of the response in unicode, and r.content is the content of the response in bytes.

In [67]:
def __page_num(PageNum=1):
    return 'https://finance.naver.com/news/mainnews.nhn?&page={}'.format(PageNum)

def light_crawling(PageNum):
    url = __page_num(PageNum)
    response = requests.get(url)
    dom = BeautifulSoup(response.content, 'lxml')
    keywords = dom.select("ul.newsList > li")
    if PageNum == 1 :
        stacked_word = [keyword.select_one('dl > dt > a').text for keyword in keywords]
        stacked_summary = [keyword.select_one('dl > dd').text for keyword in keywords]
    else : 
        stacked_word = [keyword.select_one('dl > dd.articleSubject > a').text for keyword in keywords]
        stacked_summary = [keyword.select_one('dl > dd.articleSummary').text for keyword in keywords]
#     unzip_word = [j  for x in stacked_word for j in x]
#     unzip_summary = [j  for x in stacked_summary for j in x]
#     print('Training Data set shape : ', (len(unzip_word)),str(len(unzip_summary)))
    return stacked_word , stacked_summary

In [68]:
title1 , summary1 = light_crawling(1)
title2 , summary2 = light_crawling(2)

title = title1 + title2
summary = summary1 + summary2

In [69]:
def tuning_word(word):
    new_word = str(word)
    new_word = new_word[1:-2]
    new_word = new_word.replace("'","")
    new_word = new_word.replace(',','')
    new_word = new_word.replace('  ','')
    new_word = new_word.replace('|-- ::','')
    new_word = new_word.replace('\\','')
    new_word = new_word.replace('\\t','')
    new_word = new_word.replace('\\n','')
    new_word = new_word.replace('\\xa','')
    new_word = re.sub('[0-9a-zA-Z]','',new_word)
    return new_word

In [70]:
print('before tuning: ',len(title) , len(summary))
title = tuning_word(title)
summary = tuning_word(summary)
print('after tuning: ',len(title) , len(summary))

before tuning:  26 26
after tuning:  344 1502


In [71]:
title

'코스피 방북 기대감 속 무역분쟁 경계심리 강화 "[마켓워치] 외국계 보고서 에 또 삼성전자 공매도 급증""日 태풍·지진 피해로 항공·여행株 타격 불가피" "[증시풍향계] 트럼프 대중 추가 관세 초읽기 주목" <뉴욕증시 주간전망> 관세·금리·신흥국…짙어진 먹구름 "[금융위기 년] ④긴축의 시대…금리인상에 빚폭탄 위태" [금융위기 년] ⑤벼랑 끝 신흥국…또 위기 진앙 되나 [금융위기 년] ⑥힘빠진 한국경제…저성장 구조 고착화 우려 [금융위기 년] 이창용  국장 "위기는 언제든 올 수 있다" [금융위기 년] 윤종원 靑경제수석 "양극화로 취약한 경제구조는 위험" [증시초점]코스닥 상장사들 바이오 진출 시동건 까닭은?'

In [74]:
summary = summary.replace('|-- ::','')

In [83]:
import pandas as pd
df1 = pd.DataFrame()
df1['title'] = [title]
df1

df2 = pd.DataFrame()
df2['summary'] = [summary]
df2

,summary
0,[머니투데이 진경진 기자] [[주간증시전망]코스피 전주 대비 .% 하락한 . 마감]...


In [84]:
df1.to_csv('title.csv')
df2.to_csv('summary.csv')

In [63]:
tfidv = TfidfVectorizer().fit(title)
word_vectorizer = tfidv.transform(title).toarray()

ValueError: Iterable over raw text documents expected, string object received.

In [64]:
tfidv = TfidfVectorizer().fit(summary)
summary_vectorizer = tfidv.transform(summary).toarray()

ValueError: Iterable over raw text documents expected, string object received.